In [1]:
import numpy as np
import scipy as sp
from scipy.sparse.linalg import LinearOperator

import MathFunctions as MF
import MPSOperators as MO
import SingleVUMPS as SV

In [2]:
def Calc_left_energy(AL,h,dtype):
    M = AL.shape[0]
    hl = Calc_hl(AL,h)
    _,vecR = MF.RightEigs(AL,dtype)
    VR = vecR.reshape(M,M)   VR /= np.trace(VR)
    energy = np.inner(hl.reshape(M*M),VR.reshape(M*M))
    return energy

In [3]:
def Calc_right_energy(AR,h,dtype):
    M = AR.shape[0]
    hr = Calc_hr(AR,h)
    _,vecL = MF.RightEigs(AR.transpose(2,1,0),dtype)
    VL = vecL.reshape(M,M)
    VL /= np.trace(VL)
    energy = np.inner(hr.reshape(M*M),VL.reshape(M*M))
    return energy

In [4]:
def Calc_hl(AL,h):
    ALAL = np.tensordot(AL,AL,([2],[0]))
    ALALh = np.tensordot(ALAL,h,([1,2],[0,1]))
    hl = np.tensordot(ALALh,np.conj(ALAL),([0,2,3],[0,1,2]))
    return hl

In [5]:
def Calc_hr(AR,h):
    ARAR = np.tensordot(AR,AR,([2],[0]))
    ARARh = np.tensordot(ARAR,h,([1,2],[0,1]))
    hr = np.tensordot(ARARh,np.conj(ARAR),([2,3,1],[1,2,3]))
    return hr

In [6]:
# 実数はデバッグ済み
def Calc_HL(AL,HL,h,dtype,tol=1e-17):
    M = AL.shape[0]
    hl = Calc_hl(AL,h)
    el = Calc_left_energy(AL,h,dtype)
    hl -= el * np.eye(M,M)
    #HL = Calc_HL_PowerMethod(AL,hl)
    #HL = Calc_HL_Inverse(AL,hl)
    HL = Simple_Calc_HL_Linear(AL,HL,hl,dtype,tol)
    return HL,el

In [7]:
# 実数はデバッグ済み
def Calc_HR(AR,HR,h,dtype,tol=1e-17):
    M = AR.shape[0]
    hr = Calc_hr(AR,h)
    er = Calc_right_energy(AR,h,dtype)
    hr -= er * np.eye(M,M)
    #HR = Calc_HR_PowerMethod(AR,hr)
    #HR = Calc_HR_Inverse(AR,hr)
    HR = Simple_Calc_HR_Linear(AR,HR,hr,dtype,tol)
    return HR,er

In [8]:
def Simple_Calc_HR_Linear(AR,HR,hr,dtype,tol):
    M = AR.shape[0]
    TR = np.einsum("asc,bsd -> abcd",AR,np.conj(AR)).reshape(M*M,M*M)
    _,vecL = MF.RightEigs(AR,dtype)
    VL = vecL.reshape(M,M)
    VL /= np.trace(VL)
    P = np.einsum("ab,cd -> abcd",np.eye(M),VL).reshape(M*M,M*M)
    PL = np.eye(M*M) - P
    R = np.einsum("ab,b -> a",PL,hr.reshape(M*M))
    T = np.eye(M*M) - TR + P
    HR,_ = sp.sparse.linalg.bicgstab(T,R,atol=tol,tol=tol,x0=HR.reshape(M*M))
    return HR.reshape(M,M)

In [9]:
def Simple_Calc_HL_Linear(AL,HL,hl,dtype,tol):
    M = AL.shape[0]
    TL = np.einsum("asc,bsd -> abcd",AL,np.conj(AL)).reshape(M*M,M*M)
    _,vecR = MF.RightEigs(AL.transpose(2,1,0),dtype)
    VR = vecR.reshape(M,M)
    VR /= np.trace(VR)
    P = np.einsum("ab,cd -> abcd",np.eye(M),VR).reshape(M*M,M*M)
    PR = np.eye(M*M) - P
    R = np.einsum("ab,b -> a",PR,hl.reshape(M*M))
    T = np.eye(M*M) - TL.T + P
    HL,_ = sp.sparse.linalg.bicgstab(T,R,atol=tol,tol=tol,x0=HL.reshape(M*M))
    return HL.reshape(M,M)

In [10]:
def Calc_HR_Inverse(AR,hr):
    M = AR.shape[0]
    TR = np.einsum("asc,bsd -> abcd",AR,np.conj(AR)).reshape(M*M,M*M)
    Tinf = sp.linalg.inv(np.eye(M*M) - TR)
    HR = np.einsum("ab,b -> a",Tinf,hr.reshape(M*M))
    return HR.reshape(M,M)

In [11]:
def Calc_HL_Inverse(AL,hl):
    M = AL.shape[0]
    TL = np.einsum("asc,bsd -> abcd",AL,np.conj(AL)).reshape(M*M,M*M)
    Tinf = sp.linalg.inv(np.eye(M*M) - TL)
    HL = np.einsum("a,ab -> b",hl.reshape(M*M),Tinf)
    return HL.reshape(M,M)

In [12]:
def Calc_HL_PowerMethod(AL,hl,tol=1e-15,max_iter=10000):
    M = hl.shape[0]
    HL = hl
    for i in range(max_iter):
        HL_ini = HL
        ALHL = np.tensordot(HL,AL,([0],[0]))
        TLHL = np.tensordot(ALHL,np.conj(AL),([0,1],[0,1]))
        HL = TLHL + hl
        if ( np.allclose(HL,HL_ini,rtol=0,atol=tol*M*M) ): 
            print(i)
            break
    return HL

In [13]:
def Calc_HR_PowerMethod(AR,hr,tol=1e-15,max_iter=10000):
    M = hr.shape[0]
    HR = hr
    for i in range(max_iter):
        HR_ini = HR
        ARHR = np.tensordot(AR,HR,([2],[0]))
        TRHR = np.tensordot(ARHR,np.conj(AR),([1,2],[1,2]))
        HR = TRHR + hr
        if ( np.allclose(HR,HR_ini,rtol=0,atol=tol*M*M) ): 
            print(i)
            break
    return HR

In [14]:
# 実数はデバッグ済み
def Simple_EffectiveHamiltonian_HAC(AR,AL,HR,HL,h):
    M = AR.shape[0]; D = AR.shape[1]
    Block1 = np.einsum("ghd,gia,heib,fc -> abcdef",AL,np.conj(AL),h,np.eye(M)).reshape(M*D*M,M*D*M)
    Block2 = np.einsum("fhg,cig,ehbi,da -> abcdef",AR,np.conj(AR),h,np.eye(M)).reshape(M*D*M,M*D*M)
    Block3 = np.einsum("ab,cd -> bdac",HL,np.eye(D*M)).reshape(M*D*M,M*D*M)
    Block4 = np.einsum("ab,cd -> bdac",np.eye(D*M),HR).reshape(M*D*M,M*D*M)
    return Block1 + Block2 + Block3 + Block4

In [15]:
def Simple_Next_AC(AC,AR,AL,HR,HL,h,dtype):
    M = AR.shape[0]; D = AR.shape[1]
    HAC = Simple_EffectiveHamiltonian_HAC(AR,AL,HR,HL,h)
    val,vec = sp.sparse.linalg.eigs(HAC,k=1,which="SR",v0=AC)
    if ( dtype == np.dtype("float") ): vec = vec.real
    return vec.reshape(M,D,M)

In [16]:
#　実数はデバッグ済み
def Simple_EffectiveHamiltonian_HC(AR,AL,HR,HL,h):
    M = AR.shape[0];
    Block1 = np.einsum("efc,eha,dgj,bij,fghi",AL,np.conj(AL),AR,np.conj(AR),h).reshape(M*M,M*M)
    Block2 = np.einsum("ab,cd -> bdac",HL,np.eye(M)).reshape(M*M,M*M)
    Block3 = np.einsum("ab,cd -> bdac",np.eye(M),HR).reshape(M*M,M*M)
    return Block1 + Block2 + Block3

In [17]:
def Simple_Next_C(C,AR,AL,HR,HL,h,dtype):
    M = AR.shape[0]; D = AR.shape[1]
    HC = Simple_EffectiveHamiltonian_HC(AR,AL,HR,HL,h)
    val,vec = sp.sparse.linalg.eigs(HC,k=1,which="SR",v0=C)
    if ( dtype == np.dtype("float") ): vec = vec.real
    return vec.reshape(M,M)

In [18]:
def Calc_B(AC,C,AR,AL):
    BL_square = 0
    D = AC.shape[1]
    for i in range(D):
        B = AC[:,i,:] - np.einsum("ab,bc -> ac",AL[:,i,:],C)
        BL_square += np.sum(abs(B)**2)
    BR_square = 0
    for i in range(D):
        B = AC[:,i,:] - np.einsum("ab,bc -> ac",C,AR[:,i,:])
        BR_square += np.sum(abs(B)**2)
    B_square = max(BL_square,BR_square)
    return np.sqrt(B_square)

In [19]:
D = 2; M = 10
#dtype = np.dtype("float"); A = np.random.rand(M,D,M)
dtype = np.dtype("complex"); A = np.random.rand(M,D,M) + 1j * np.random.rand(M,D,M); 
Sx,Sy,Sz,Su,Sd = MF.Spin(D)
h = - ( np.kron(Sx,Sx) + np.kron(Sy,Sy) - np.kron(Sz,Sz) ).real.reshape(D,D,D,D)
AC,C,AR,AL = MO.MixedCanonicalForm(A,dtype)
HR = np.random.rand(M,M); HL = np.random.rand(M,M) # initial HR and HL
for i in range (30):
    HR,er = SV.Calc_HR(AR,HR,h,dtype)
    HL,el = SV.Calc_HL(AL,HL,h,dtype)
    AC = SV.Simple_Next_AC(AC,AR,AL,HR,HL,h,dtype)
    C = SV.Simple_Next_C(C,AR,AL,HR,HL,h,dtype)
    AR = SV.Next_AR_SVD(AC,C)
    #AR = SV.Next_AR_PolarDecomposition(AC,C)
    AL = SV.Next_AL_SVD(AC,C)
    #AL = SV.Next_AL_PolarDecomposition(AC,C)
    B = SV.Calc_B(AC,C,AR,AL)
    print (er,el,B)

(-0.24600543459895857-1.179069862577986e-17j) (-0.24600543459895785-1.5611494844254559e-16j) 0.12287984516477904
(-0.4379123554061504-1.9081958235744878e-17j) (-0.4368158550016288+1.734723475976807e-17j) 0.018602503146690454
(-0.4423642711114629+9.020562075079397e-17j) (-0.4423314614347525+9.71445146547012e-17j) 0.0055443551515646965
(-0.4428169810564584+1.6740081543176188e-16j) (-0.44282047337576735+0j) 0.0029857411971755455
(-0.4429436788518709+2.6020852139652106e-17j) (-0.44294441951208097+7.632783294297951e-17j) 0.0016979025475032763
(-0.4429838769136504-2.949029909160572e-17j) (-0.44298400448990616+3.122502256758253e-17j) 0.000986565881597179
(-0.44299748152027274-2.654126918244515e-16j) (-0.4429975024867074+2.42861286636753e-17j) 0.0005774050574142657
(-0.443002182729303-5.898059818321144e-17j) (-0.4430021862401964+2.220446049250313e-16j) 0.0003385378321704098
(-0.44300381386839116+2.7755575615628914e-17j) (-0.44300381443314274+3.8163916471489756e-17j) 0.0001985849579294476
(-0.4

In [20]:
def Tsum(T,R):
    Rini = R
    for i in range (10000):
        #print ( T.shape, R.shape)
        T2 = np.einsum("ab,b -> a",T,R)
        R = T2 + Rini
    return R

In [19]:
def Calc_HR2(A,hr_tilde,iteration = 10000):
    R = hr_tilde
    for i in range (iteration):
        R_next = np.tensordot(np.tensordot(A,R,([2],[0])),A,([0,2],[0,2])) + hr_tilde
        if abs(np.sum((R_next - R)**2)) < 1e-11:
            break
        R = R_next
    return R_next   

In [63]:
def Calc_HL2(A,hl_tilde,iteration = 10000):
    L = hl_tilde
    for i in range (iteration):
        L_next = np.tensordot(np.tensordot(L,A,([0],[1])),A,([0,1],[1,0])) + hl_tilde
        if abs(np.sum((L_next - L)*2)) < 1e-11:
            break
        L = L_next
    return L_next

In [29]:
A = np.random.rand(M,D,M)
dtype = np.dtype("float");
h = ( np.kron(Sx,Sx) + np.kron(Sy,Sy) + np.kron(Sz,Sz) ).real.reshape(D,D,D,D)
AC,C,AR,AL = MO.MixedCanonicalForm(A,dtype)
hr = Calc_hr(AR,h)
er = Calc_right_energy(AR,h,dtype)
hr -= er * np.eye(M,M)
HR = Calc_HR_PowerMethod(AR,hr)
HR2 = Calc_HR2(AR.transpose(1,0,2),hr)
print (np.allclose(HR,HR2))
#print (HR.reshape(M*M)-HR2,HR2-HR3,HR3-HR.reshape(M*M),HR3)

15


NameError: name 'Calc_HR2' is not defined